# <center>제2고지 자연스러운 코드로</center>

In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

# 11단계 가변 길이 인수(순전파 편)

DeZero가 인수 또는 반환값의 수가 달라질 수 있는, 가변 길이 입출력을 처리할 수 있도록 확장  
  
가변 길이 입출력을 표현하기위해 변수들을 리스트에 넣어 처리하면  
전처럼 Function 클래스가 '하나의 인수'만 받고 '하나의 값'만 반환  
  
Function 클래스의 \_\_call__ 메서드 인수, 반환값을 리스트로 수정

In [2]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

덧셈을 수행하는 Add 클래스의 forward 메서드 구현  
인수는 변수가 두 개 담긴 리스트  
결과는 튜플로 반환

In [7]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [4]:
xs = [Variable(np.array(2)), Variable(np.array(3))] # 리스트로 준비
f = Add()
ys = f(xs) # ys 튜플
y = ys[0]
print(y.data)

5


# 12단계 가변 길이 인수(개선 편)

### 첫 번째 개선

리스트나 튜플을 거치지 않고 인수와 결과를 직접 주고받도록 Function 클래스 수정

In [5]:
class Function:
    def __call__(self, *inputs): # 별표를 붙임
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

인수 앞에 별표(\*)를 붙이면 리스트를 사용하는 대신 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있음

In [6]:
def f(*x):
    print(x)
    
f(1,2,3)

f(1,2,3,4,5,6)

(1, 2, 3)
(1, 2, 3, 4, 5, 6)


In [8]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 두 번째 개선

입력도 변수를 직접 받고 결과도 변수를 직접 돌려주도록 수정

In [9]:
class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # 별표를 붙여 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 추가 지원
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

~~~python
self.forward(*xs)
~~~  
  
에서 함수를 호출할 때 별표를 붙였는데, 이렇게 하면 리스트 언팩이 이뤄짐  
리스트의 원소를 낱개로 풀어서 전달함  
ex) 
xs = [x0, x1]   
self.forward(\*xs)를 하면  
self.forward(x0, x1)로 호출하는 것과 동일하게 동작함  
  
Add 클래스도 수정

In [10]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

Add 클래스를 파이썬 함수로 사용할 수 있는 코드를 추가

In [11]:
def add(x0, x1):
    return Add()(x0, x1)

In [12]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) # Add 클래스 생성 과정이 감춰짐
print(y.data)

5


# 13단계 가변 길이 인수(역전파 편)

<img src='./img/2/add.png' width=500>

위의 그림과 같이 덧셈의 순전파는 입력이 2개, 출력이 1개이고,  
반대로 역전파는 입력이 1개, 출력이 2개임  
  
$y = x_0+x_1$과 같이 입력 변수가 여러 개인 함수를 다변수 함수라고 하며,  
다변수 함수에서 하나의 입력 변수에만 주목하여(다른 변수는 상수로 취급) 미분하는 것을 편미분이라고 함  
  
덧셈의 역전파는 출력 쪽에서 전해지는 미분값에 1을 곱한 값이 입력 변수의 미분  
즉, 역전파는 상류에서 흘러오는 미분값을 그대로 흘려보냄  
  
이를 반영하여 Add 클래스에 backward 메서드를 구현

In [92]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy): # 입력 1개
        return gy, gy # 출력 2개

여러 개의 값을 반환할 수 있도록 Variable 클래스의 backward 메서드 수정

In [14]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] # outputs에 담겨있는 미분값들을 리스트에 담음
            gxs = f.backward(*gys) # f의 역전파 호출. 리스트 언팩
            if not isinstance(gxs, tuple): # gxs가 튜플이 아니면 튜플로 변환
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # f.inputs와 gxs의 각 원소는 서로 대응 관계
                x.grad = gx # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
            
                if x.creator is not None:
                    funcs.append(x.creator)

Square 클래스도 수정한 Variable 클래스와 Function 클래스에 맞게 수정

In [51]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [52]:
def square(x):
    return Square()(x)

In [17]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


# 14단계 같은 변수 반복 사용

<img src='./img/2/ex_1.png' width=400>  
  
현재의 DeZero는 위의 그림과 같이 같은 변수를 반복해서 사용할 경우 의도대로 동작하지 않을 수 있음

In [18]:
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


위 계산에서 미분값은 2가 되야 하는데,  
현재 구현에서는 출력 쪽에서 전해지는 미분값을 그대로 대입하므로  
같은 변수를 반복해서 사용하면 전파되는 미분값이 덮어 써짐  
  
따라서 Variable 클래스를 수정

In [19]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx # 처음 미분값을 설정하는 경우에는 기존과 같이 출력 쪽에서 전해지는 미분값을 그대로 대입
                else:
                    x.grad = x.grad + gx # 다음번부터는 전달된 미분값을 더해줌
                    # x.grad += gx 라고 쓰면 문제 발생 -> 부록 A 참고
            
                if x.creator is not None:
                    funcs.append(x.creator)

In [20]:
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 2.0


In [21]:
x = Variable(np.array(3.0))
y = add(add(x,x),x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 9.0
x.grad 3.0


여전히 문제가 있음  
같은 변수를 사용하여 '다른 계산'을 할 경우 계산이 꼬임

In [22]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
y = add(add(x,x),x)
y.backward()
print(x.grad) # 3이 나와야 함

2.0
5.0


위 예시와 같이 Variable 인스턴스인 x를 재사용한 경우 첫 번째 미분값이 더해져서 나옴  
  
이 문제를 해결하기 위해 Variable 클래스에 미분값을 초기화하는 cleargrad 메서드를 추가  
이 메서드를 사용하면 여러 가지 미분을 연달아 계산할 때 똑같은 변수를 재사용 할 수 있음

In [23]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)
                    
    def cleargrad(self): # 추가
        self.grad = None

In [24]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad() # 미분값 초기화
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0


이 cleargrad 메서드는 최적화 문제(함수의 최솟값과 최댓값을 찾는 문제)를 풀 때 유용하게 사용할 수 있음!

# 15단계 복잡한 계산 그래프(이론 편)

<img src='./img/2/ex_2.png' width=800>  
  
지금까지는 일직선의 계산만 해서 문제가 없었으나, 위 그림과 같이 복잡한 계산을 하게 되면 또 문제가 발생함🥲  

  
<img src='./img/2/ex_3.png' width=400>  
  
변수를 반복해서 사용하면 역전파 때는 출력 쪽에서 전파되는 미분값을 더해야 하므로, 계산 중간에 등장하는 변수 a의 경우 a의 출력 쪽에서 전파하는 2개의 미분값이 필요함  
즉, 함수 B와 C의 역전파를 모두 끝내고 나서 함수 A를 역전파해야함  
따라서 올바른 역전파의 순서는 위의 그림과 같다.  
  
<img src='./img/2/ex_4.png' width=400>  
  
하지만, 현재 구현한 Dezero는 위 그림처럼 함수 처리 순서도 잘못되었고, 함수 A의 역전파가 두 번 일어남  
funcs 리스트에 창조자를 추가하고 pop함수를 사용하여 마지막 원소를 꺼내는 방식으로 진행되기 때문!  
  
이는 리스트에서 출력 쪽에 더 가까운 함수를 꺼낼 수 있도록 함수에 '우선순위'를 주는 방식으로 해결할 수 있음  
  
위상 정렬(Topological Sort) 알고리즘을 사용하면 노드의 연결 방법을 기초로 노드들을 정렬할 수 있고, 그 정렬 순서를 우선순위로 사용해도 되지만, 더 쉬운 방법이 있음  
  
<img src='./img/2/ex_5.png' width=500>  
  
순전파 때 '함수'가 '변수'를 만들어내는 과정에서 볼 수 있는 '창조자-피조물 관계' 혹은 '부모-자식 관계'를 이용하여 함수와 변수의 '세대(generation)'을 기록하고,  
이 '세대'를 우선순위로 사용하여 세대 수가 큰 쪽부터 처리하면 '부모'보다 '자식'을 먼저 처리할 수 있음

# 16단계 복잡한 계산 그래프(구현 편)

순전파 시 '세대'를 설정하고  
역전파 시 최근 세대의 함수부터 꺼내도록 Variable 클래스와 Function 클래스에 인스턴스 변수 generation(몇 번째 '세대'의 함수(혹은 변수)인지 나타내는 변수)을 추가  
  
먼저 Variable 클래스는 generation을 0으로 초기화하고,  
set_creator 메서드가 호출될 때 부모 함수의 세대보다 1만큼 큰 값을 설정함

In [25]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록함(부모 세대 + 1)
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)
                    
    def cleargrad(self): 
        self.grad = None

<img src='./img/2/ex_6.png' width=500>

Function 클래스의 generation은 입력 변수와 같은 값으로 설정  
  
위 예시처럼 입력 변수의 generation이 4라면 함수의 generation도 4가 됨  
입력 변수가 둘 이상이라면 가장 큰 generation의 수를 선택함

In [26]:
class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) # generation 설정
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

함수를 세대 순으로 꺼내기 위해서 더미(dummy) DeZero 함수를 사용

In [27]:
generations = [2,0,1,4,2]
funcs = []

for g in generations:
    f = Function() # 더미 함수 클래스
    f.generation = g
    funcs.append(f)
    
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

위와 같이 더미 함수를 준비하고 funcs 리스트에 추가한 후, 이 리스트에서 세대가 가장 큰 함수를 꺼냄

In [28]:
funcs.sort(key=lambda x: x.generation) # generation 기준으로 리스트 오름차순 정렬
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [29]:
f = funcs.pop()
f.generation

4

< 참고 > 책에서 다루진 않지만 '우선순위 큐'를 이용하면 모든 원소를 정렬할 필요 없이 더 효율적

Variable 클래스의 backward 메서드 수정

In [30]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
# ----------------------------------------------------------------
        funcs = [] 
        seen_set = set() # funcs 리스트에 같은 함수를 중복 추가하는 일을 막기 위해 집합을 이용
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
# ----------------------------------------------------------------
        
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) # 수정
                    
    def cleargrad(self): 
        self.grad = None

여기서, add_func 함수를 backward 메서드 안에 중첩 함수로 정의했음  
중첩 함수는 주로 다음 두 조건을 충족할 때 적합함  
- 감싸는 메서드(backward 메서드) 안에서만 이용한다.
- 감싸는 메서드(backward 메서드)에 정의된 변수(funcs과 seen_set)를 사용해야 한다.

In [35]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


# 17단계 메모리 관리와 순환 참조

'파이썬'이라고 하면 보통은 '프로그래밍 언어'를 가리키지만 파이썬 코드를 실행하는 '프로그램'을 지칭할 때도 사용함  
이 프로글매을 일반적으로 '파이썬 인터프리터'라고 부르며,  
표준으로 사용되는 파이썬 인터프리터는 C언어로 구현된 CPython임  
이번 단계에서 설명하는 파이썬 메모리 관련 설명은 CPython 기준!

파이썬은 필요 없어진 객체를 메모리에서 자동으로 삭제함  
  
파이썬(CPython)의 메모리 관리는 다음 두 가지 방식으로 진행됨  
- 참조 카운트 : 참고(reference) 수를 세는 방식
- GC(Garbage Collection) : 세대(generation)를 기준으로 쓸모없어진 객체(garbage)를 회수(collection)하는 방식  

파이썬 메미로 관리의 기본은 참조 카운트  
참조 카운트는 구조가 간단하고 속도도 빠름  
모든 객체는 참조 카운트가 0인 상태로 생성되고,  
다른 객체가 참조할 때마다 1씩 증가  
반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 파이썬 인터프리터가 회수함  
객체가 더 이상 필요 없어지면 즉시 메모리에서 삭제됨  
  
참조 카운트 증가하는 경우  
- 대입 연산자를 사용할 때
- 함수에 인수로 전달할 때
- 컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가할 때  
  
ex)  
  
~~~python
class obj():
    pass

def f(x):
    print(X)
    
a = obj()   # 변수에 대입 : 참조 카운트 1
f(a)        # 함수에 전달 : 함수 안에서는 참조 카운트 2
            # 함수 완료 : 빠져나오면 참조 카운트 1
a = None    # 대입 해제 : 참조 카운트 0(아무도 참조하지 않는 상태) -> 이렇게 0이 되는 즉시 해당 객체는 메모리에서 삭제됨
~~~  

ex)  
  
<img src='./img/2/memory_1.png' width=500>

~~~python 
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c

a = b = c = None
~~~

하지만 참조 카운트로는 순환 참조(circular reference)를 해결할 수 없음  
  
<img src='./img/2/memory_2.png' width=500>  
  
~~~python
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a  

a = b = c = None
~~~  
  
위 예시처럼 객체 서로가 서로를 참조하는 것을 순환 참조라고 함  
이 때 오른쪽 그림을 보면 'a = b = c = None'을 실행했음에도 a, b, c의 참조 카운트가 (0이 아닌) 모두 1이고, 사용자가 이 객체 중 어느 것에도 접근할 수 없음. 결과적으로 메모리에서 삭제되지 않음  
  
그래서 세대별 가비지 컬렉션(generation garbage collection, GC)가 등장함  
GC는 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출됨  
(gc 모듈을 임포트해서 gc.collect()를 실행하여 명시적으로 호출할 수도 있음)  
GC는 순환 참조를 올바르게 처리함  
신경망에서 메모리는 중요하기 대문에 순환 참조를 만들지 않는 것이 좋음  
  
하지만, DeZero에서는 '변수'와 '함수'를 연결하는 방식에 순환 참조가 있음  
이는 weakref.ref 함수를 사용하여 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능을 하는 약한 참조(weak reference)를 만들어 해결할 수 있음

In [36]:
import weakref
import numpy as np

a = np.array([1,2,3])
b = weakref.ref(a)

b # 약한 참조

<weakref at 0x7f8520031cc0; to 'numpy.ndarray' at 0x7f853005f0f0>

In [37]:
b() # 참조된 데이터에 접근

array([1, 2, 3])

In [38]:
a = None
b

<weakref at 0x7f8520031cc0; to 'numpy.ndarray' at 0x7f853005f0f0>

In [39]:
b # 파이썬 인터프리터에서는 dead라고 나옴

<weakref at 0x7f8520031cc0; to 'numpy.ndarray' at 0x7f853005f0f0>

a = None을 실행하면 ndarray 인스턴스는 참조 카운트 방식에 따라 메모리에서 삭제되고,  
b도 참조를 가지고 있지만 약한 참조이기 때문에 dead라는 문자가 나오고, 이로써 ndarray 인스턴스가 삭제되었음을 알 수 있음  
  
다만, IPython과 주피터 노트북 등의 인터프리터는 인터프리터 자체가 사용자가 모르는 참조를 추가로 유지하기 때문에 b가 여전히 유효한 참조를 유지함  
위 코드는 파이썬 인터프리터에서 실행한다고 가정하였음

위에서 설명한 weakref 구조를 Function 클래스와 Variable 클래스에 추가

In [40]:
import weakref

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] # 약한 참조
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [41]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs] # 참조된 데이터에 접근
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
    def cleargrad(self): 
        self.grad = None

In [44]:
# 동작 확인
for i in range(10):
    x = Variable(np.random.randn(10000)) # 거대한 데이터
    y = square(square(square(x))) # 복잡한 계산 수행

<img src='./img/2/memory_3.png' width=700>

for 문이 두 번째 반복될 때 x와 y가 덮어 써짐  
사용자는 이전의 계산 그래프를 더 이상 참조하지 않게 됨  
참조 카운트가 0이 되고 이 시점에 계산 그래프에 사용된 메모리가 바로 삭제됨

# 18단계 메모리 절약 모드

DeZero의 메모리 사용을 개선할 수 있는 구조 두 가지를 도입  
- 역전파 시 사용하는 메모리 양을 줄이는 방법으로, 불필요한 미분 결과를 보관하지 않고 즉시 삭제  
- '역전파가 필요 없는 경우용 모드'를 제공. 이 모드에서는 불필요한 계산을 생략

### 첫 번째 개선  
__불필요한 미분 결과를 보관하지 않고 즉시 삭제__  
  
  
현재의 DeZero는 모든 변수가 미분값을 변수에 저장해두고 있음

In [46]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


위 예시에서 사용자가 제공한 변수는 x0, x1이고,  
t와 y는 계산 결과로 만들어지며,  
y.backward()를 실행하면 모든 변수가 미분 결과를 메모리에 유지함  
하지만 많은 경우, 역전파로 구하고 싶은 미분값은 말단 변수(x0, x1)뿐이기 때문에  
중간 변수의 미분값은 필요하지 않음  
  
따라서 Variable 클래스의 backward 메서드에 중간 변수에 대해서는 미분값을 제거하는 모드를 추가

In [47]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): # retain_grad 추가
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad: # retain_grad가 False면 중간변수 미분값을 모두 None으로 재설정
                for y in f.outputs:
                    y().grad = None # y는 약한 참조(weakref)기 때문에 이 코드를 실행하면 참조 카운트가 0이 되어 메모리에서 삭제됨
    def cleargrad(self): 
        self.grad = None

In [48]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


### 두번 째 개선  
__불필요한 계산을 생략(역전파가 필요없는 경우)__  
  
인스턴스 변수 inputs는 역전파 계산 시 사용됨  
신경망 학습 시에는 미분값이 필요하지만, 추론 시에는 단순히 순전파만 하기 때문에 미분값이 필요 없으며, 중간 결과를 곧바로 버리면 메모리 사용량을 크게 줄일 수 있음  
  
따라서 Config 클래스를 만들어 활용  

In [49]:
class Config:
    enable_backprop = True

True인 경우 '역전파 활성 모드'

Function 클래스에서 Config를 참조하게 하여 모드를 전환할 수 있도록 수정

In [50]:
import weakref

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: # 추가(True일 경우 역전파 활성)
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs] 
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

모드 전환

In [53]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False # 중간 결과는 사용 후 곧바로 삭제됨(다른 객체에서의 참조가 없어지는 시점에 삭제됨)
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

AttributeError: 'NoneType' object has no attribute 'generation'

In [54]:
Config.enable_backprop = True # 다시 돌려 놓기!

with 문은 with 블록에 들어갈 때 파일이 열리고, with 블록 안에서 파일은 계속 열린 상태이다가 블록을 빠져나올 때 자동으로 닫힘  
  
__with문의 원리를 이용하여 '역전파 비활성 모드'로 전환__  
  
ex)  
~~~python
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
~~~
  
위의 코드는 with using_config('enable_backprop', False): 안에서만 '역전파 비활성 모드가 되고  
with 블록을 벗어나면 역전파 활성 모드로 돌아감  

또한 다음과 같이 contextlib 모듈을 사용할 수 있음
  
~~~python
import contextlib

@contextlib.contextmanager
def config_test():
    print('start') # 전처리
    try:
        yield
    finally:
        print('done') # 후처리
        
with config_test():
    print('process...')  
~~~
  
\@contextlib.contextmanager 데코레이터를 달면 문맥을 판단하는 함수가 만들어짐
이 하수 안에서 yield 전에는 전처리 로직을, 후에는 후처리 로직을 작성함  
이렇게 with config_test(): 형태의 구문을 사용할 수 있음  
  
with 블록 안에서 예외가 발생한 경우 yield를 실행하는 코드로도 전달됨  
따라서 yield는 try/finally로 감싸야 함

In [55]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

- getattr(object, name[, default])
    : object의 속성 값을 가져옴  
  
- setattr(object, name, value)
    : object에 새로운 값을 설정

with 블록에 들어갈 때 name으로 지정한 Config 클래스 속성이 value로 설정되고  
with 블록을 빠져나오면서 원래 값(old_value)로 다시 복원됨

위 함수는 아래와 같이 사용할 수 있으며, 역전파가 필요 없는 경우에는 with 블록에서 순전파 코드만 실행

In [56]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

with using_config('enable_backprop', False): 를 더 간략하게 사용하기 위해 다음 함수를 작성

In [57]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

# 19단계 변수 사용성 개선

여러 수많은 변수를 처리해야 하기 때문에 변수에 '이름' 붙여주도록 Variable 클래스에 name이라는 인스턴스 변수를 추가 

In [58]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.name = name # 추가
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): # retain_grad 추가
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 
    def cleargrad(self): 
        self.grad = None

Variable은 데이터를 담는 '상자' 역할이지만,  
Variable이 데이터인 것처럼 보이게 투명화하는 장치를 추가!  

(Variable안에 있는 ndarray를 ndarray처럼 사용할 수 있도록)  

In [59]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.name = name 
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property # 추가
    def shape(self): 
        return self.data.shape

    @property # 추가
    def ndim(self):
        return self.data.ndim

    @property # 추가
    def size(self):
        return self.data.size

    @property # 추가
    def dtype(self):
        return self.data.dtype
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 
    def cleargrad(self): 
        self.grad = None

이제 shape, ndim, size, dtype 메서드를 인스턴스 변수처럼 사용할 수 있게 됨!

In [60]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape) # x.shape() 대신 x.shape으로 호출할 수 있음

(2, 3)


len 함수와 Variable의 내용을 쉽게 확인할 수 있도록 print 함수를 사용하여 데이터 내용을 출력하는 함수 추가
  

In [61]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.name = name 
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def shape(self): 
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): # 추가
        return len(self.data)

    def __repr__(self): # 추가
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) # 보기 좋게 출력하기 위해 전처리
        return 'variable(' + p + ')'
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 
    def cleargrad(self): 
        self.grad = None

In [62]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(len(x)) # len 함수는 ndarray 인스턴스의 경우 첫 번째 차원의 원소 수를 반환함

2


In [63]:
print(repr(x))

variable([[1 2 3]
          [4 5 6]])


# 20단계 연산자 오버로드(1)

__+와 \* 연산자를 지원하도록 Variable 클래스를 확장__

<img src='./img/2/mul_1.png' width=500>  
  
$y=x_0\times x_1$일 때 곱셈의 미분은 $\frac{\partial{y}}{\partial{x_0}}=x_1$, $\frac{\partial{y}}{\partial{x_1}}=x_0$  
따라서 역전파는 위 그림과 같이 이뤄짐

In [93]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

Mul 클래스를 파이썬 함수로

In [65]:
def mul(x0, x1):
    return Mul()(x0, x1)

In [66]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


연산자를 오버로드(operator overload)하면 +와 \*같은 연산자 사용 시 사용자가 설정한 함수가 설정됨  
파이썬에서는 \_\_add__와 \_\_mul__ 같은 특수 메서드를 정의함으로써 사용자 지정 함수가 호출되도록 함  
  
Variable 클래스에 연산자 +, * 연산자 오버로드  
  
<img src='./img/2/mul_2.png' width=300>  
  
~~~python
class Variable:
    ...
    
    def __mul__(self, other):
        return mul(self, other)
~~~  

위와 같이 Variable 클래스에 __mul__ 메서드를 추가해서 \*를 사용한 곱셈을 할 수 있음  
더 간단히 처리하려면 다음 코드와 같음

In [84]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.name = name 
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def shape(self): 
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)

    def __repr__(self): 
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) 
        return 'variable(' + p + ')'
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 
    def cleargrad(self): 
        self.grad = None
        
Variable.__mul__ = mul # 추가
Variable.__add__ = add # 추가

위와 같이 Variable 클래스를 정의한 후 아래 코드를 작성하면  
파이썬에서는 함수도 객체이므로 함수 자체를 할당할 수 있음  
  
이렇게 +와 \* 연산자 오버로드를 함

In [68]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


# 21단계 연산자 오버로드(2)

__Variable 인스턴스와 ndarray 인스턴스, int나 float 등을 함께 사용할 수 있도록 수정__  
  
예를 들어 a가 Variable 인스턴스일 때,  
a * np.array(2)라는 코드가 나오면 ndarray인스턴스를 자동으로 Variable 인스턴스로 변환해주는 as_variable 함수 구현

In [69]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

Function 클래스의 \_\_call__ 메서드에 as_variable 함수를 이용하도록 코드 추가

In [89]:
import weakref

class Function:
    def __call__(self, *inputs): 
        inputs = [as_variable(x) for x in inputs] # 추가
        
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: 
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs] 
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [90]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


파이썬의 float와 int, np.float64와 np.int64 같은 타입과도 함께 사용할 수 있도록 수정

In [91]:
abc = 3
abc1 = as_array(abc)
print(type(abc1))

<class 'numpy.ndarray'>


In [94]:
def add(x0, x1):
    x1 = as_array(x1) # 9단계에서 구현한 함수 # ndarray 인스턴스는 Function 클래스에서 Variable 인스턴스로 변환됨
    return Add()(x0, x1)

In [95]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [96]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


하지만 위의 방식에는 두 가지 문제가 있음

__문제 1 : 첫 번째 인수가 float나 int인 경우__

In [79]:
y = 2.0 * x # 오류 발생

TypeError: unsupported operand type(s) for *: 'float' and 'Variable'

오류가 발생하는 과정은 다음과 같음  
  
1. 연산자 왼쪽에 있는 2.0의 \_\_mul__ 메서드를 호출하려 시도함
2. 하지만 2.0은 float 타입이므로 \_\_mul__ 머세드는 구현되어 있지 않음
3. 다음은 * 연산자 오른쪽에 있는 x의 특수 메서드를 호출하려고 시도함
4. x가 오른쪽에 있기 때문에 \_\_rmul__ 메서드를 호출하려 시도함
5. 하지만 Variable 인스턴스에는 \_\_rmul__ 메서드가 구현되어 있지 않음

곱셈의 경우 피연산자가 좌항이면 \_\_mul__ 메서드가 호출되고,  
우항이면 \_\_rmul__ 메서드가 호출됨  
  
<img src='./img/2/mul_3.png' width=300>  
  
위 그림과 같이 \_\_rmul__(self, other)의 인수 중 self는 자신인 x에 대응하고, other은 다른 쪽 항인 2.0에 대응함  
곰셈과 덧셈의 경우 좌항과 우항을 바꿔도 결과가 같기 때문에 둘을 구별할 필요가 없음  
따라서 특수 메서드를 다음과 같이 설정

In [102]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [98]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)

variable(7.0)


__문제2 : 좌항이 ndarray 인스턴스인 경우__  

In [99]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x

이런 경우 좌항인 ndarray 인스턴스의 \_\_add__ 메서드가 호출됨  
우항인 Variable 인스턴스의 \_\_radd__ 메서드가 호출되도록 하려면 연산자 우선순위를 지정해야 함  
  
Variable 인스턴스 속석에 \_\_array_priority__를 추가하고 큰 정수로 설정하면 Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스의 연산자 우선순위보다 높일 수 있음  
좌항이 ndarray라 해도 우항인 Variable 인스턴스의 연산자 메서드가 우선적으로 호출됨

In [100]:
class Variable:
    def __init__(self, data, name=None):
        __array_priority__ = 200 # 큰 정수로 설정
        
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.name = name 
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def shape(self): 
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)

    def __repr__(self): 
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) 
        return 'variable(' + p + ')'
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad=False): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [] 
        seen_set = set() 
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator) 
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator) 
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 
    def cleargrad(self): 
        self.grad = None

In [103]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
y

array([variable([3.])], dtype=object)

# 22단계 연산자 오버로드(3)

연산자 추가  
  
특수 메서드|예
--------|--
\_\_neg__(self)|-self
\_\_sub__(self, other)|self - other
\_\_rsub__(self, other)|other - self
\_\_truediv__(self, other)|self / other
\_\_rtruediv__(self, other)|other / self
\_\_pow__(self, other)|self ** other  
  
새로운 연산자 추가하는 순서  
  
1. Function 클래스를 상속하여 원하는 함수 클래스를 구현함(ex: Mul클래스)
2. 파이썬 함수로 사용할 수 있도록 함(ex: mul함수)
3. Variable 클래스의 연산자를 오버로드함(ex: Variable.\_\_mul__ = mul)

__음수(부호 변환)__  
  
$y=-x$일 때 음수의 미분값은 $\frac{\partial{y}}{\partial{x}}=-1$  
따라서 역전파는 상류(출력 쪽)에서 전해지는 미분에 -1을 곱하여 하류로 흘려줌

In [104]:
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [105]:
x = Variable(np.array(2.0))
y = -x # 부호를 바꾼다.
print(y)

variable(-2.0)


__뺄셈__  
  
$y=x_0-x_1$일 때 뺄셈의 미분값은 $\frac{\partial{y}}{\partial{x_0}}=1$, $\frac{\partial{y}}{\partial{x_1}}=-1$  
따라서 역전파는 상류(출력 쪽)에서 전해지는 미분에 1을 곱한 값이 $x_0$의 미분 결과,  
-1을 곱한 값이 $x_1$의 미분 결과가 됨

In [106]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

<img src='./img/2/sub_1.png' width=300>

예를 들어 y = 2.0 - x 같이 $x_0$이 Variable 인스턴스가 아닌 경우에는 제대로 처리가 되지 않음  
  
뺄셈에서는 좌우를 구별해야하므로 우항을 대상으로 할 경우 따로 함수를 준비해야 함  

In [107]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0) # x0과 x1의 순서를 바꿈

Variable.__rsub__ = rsub

In [108]:
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0

print(y1, y2)

variable(0.0) variable(1.0)


__나눗셈__  
  
$y=x_0/x_1$일 때 나눗셈의 미분값은 $\frac{\partial{y}}{\partial{x_0}}=\frac{1}{x_1}$, $\frac{\partial{y}}{\partial{x_1}}=-\frac{x_0}{(x_1)^2}$  
  
나눗셈도 뺄셈과 마찬가지로 좌우를 구별해야 함

In [109]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1**2)
        return gx0, gx1
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0) # x0과 x1을 바꾼다.

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [110]:
x = Variable(np.array(12.0))
y1 = 24.0 / x
y2 = x / 3.0

print(y1, y2)

variable(2.0) variable(4.0)


__거듭제곱__  
  
$y=x^c$일 때 거듭제곱의 미분값은 $\frac{\partial{y}}{\partial{x}}=cx^{c-1}$  
  

In [111]:
class Pow(Function):
    def __init__(self, c):
        self.c = c
        
    def forward(self, x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c-1) * gy
        return gx
    
def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [112]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)


# 23단계 패키지로 정리

- 모듈 : 다른 파이썬 프로그램에서 임포트(import)하여 사용하는 것을 가정하고 만들어진 파이썬 파일 
- 패키지 : 여러 모듈을 묶은 것. 디렉터리를 만들고 그 안에 모듈(파이썬 파일)을 추가
- 라이브러리 : 여러 패키지를 묶은 것. 하나 이상의 디렉터리로 구성됨. 때로는 패키지를 라이브러리라고 부르기도 함

패키지로 정리하기  
  
__1. 파일 구성__  
  
dezero라는 공통의 디렉터리 생성  
  
__2. 코어 클래스로 옮기기__  
  
step22.py에 정의된 다음 클래스들과 함수들을 코어 파일로 복사(파일명은 core_simple.py)  
- 클래스 : Config, Variable, Function, Add(Function), Mul(Function), Neg(Fungtion), Sub(Function), Div(Function), Pow(Function)  
- 함수 : using_config, no_grad, as_array, as_variable, add, mul, neg, sub, rsub, div, rdiv, pow  
  
__3. 연산자 오버로드__  
  
~~~python
def setup_variable():
    Variable.__add__ = add
    Variable.__radd__ = add
    Variable.__mul__ = mul
    Variable.__rmul__ = mul
    Variable.__neg__ = neg
    Variable.__sub__ = sub
    Variable.__rsub__ = rsub
    Variable.__truediv__ = div
    Variable.__rtruediv__ = rdiv
    Variable.__pow__ = pow
~~~
위 코드를 core_simple.py에 추가  
이 함수를 호출하면 Variable의 연산자들이 설정됨  

이 함수는 dezero/\_\_init__.py에서 호출!  
\_\_init__.py는 모듈을 임포트할 때 가장 먼저 실행되는 파일임  
따라서 dezero/\_\_init__.py에 다음 코드 작성

~~~python
from dezero.core_simple import Variable
from dezero.core_simple import Function
from dezero.core_simple import using_config
from dezero.core_simple import no_grad
from dezero.core_simple import as_array
from dezero.core_simple import as_variable
from dezero.core_simple import setup_variable

setup_variable()
~~~
  
from dezero.core_simple import (  ) -> 이 코드로 인해  
dezero를 이용하는 사용자의 코드는 from dezero import (  ) 와 같이 짧아짐  

33단계부터는 core_simple.py가 아닌 core.py로 대체할 예정  
따라서 \_\_init__.py 파일을 다음과 같이 수정  
  
~~~python
is_simple_core = True

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable

else:
    from dezero.core import Variable
    from dezero.core import Function
    from dezero.core import using_config
    from dezero.core import no_grad
    from dezero.core import as_array
    from dezero.core import as_variable
    from dezero.core import setup_variable
    
setup_variable()
~~~
  
(33단계부터 is_simple_core의 값을 False로 바꿔주기!)  
  
이렇게 dezero라는 패키지가 만들어졌음

In [113]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.dirname(__file__), '..') # 파이썬 명령어를 어디에서 실행하든 deero 디렉터리 파일들을 임포트 할 수 있음
    
import numpy as np
from dezero.core_simple import Variable, setup_variable

setup_variable()

x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

variable(16.0)
8.0


# 24단계 복잡한 함수의 미분

<img src='./img/2/test.png' width=500>
  
최적화 문제에서 자주 사용되는 테스트 함수로 Dezero가 미분을 얼마나 잘 계산하는지 확인  

__Sphere 함수__  
  
$z =x^2+y^2$  

In [114]:
import numpy as np
from dezero import Variable

def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)
z.backward()
print(x.grad, y.grad)

2.0 2.0


__matyas 함수__  
  
$z=0.26(x^2+y^2)-0.48xy$

In [115]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()
print(x.grad, y.grad)

0.040000000000000036 0.040000000000000036


__Goldstein-Price 함수__  
  
$f(x,y)=[1+(x+y+1)^2(19-14x+3x^2-14y+6xy+3y^2)]\
    [30+(2x-3y)^2(18-32x+12x^2+48y-36xy+27y^2)]$

In [116]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2))*\
    (30 + (2*x - 3*y)**2 * (18 -32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

In [117]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0


# APPENDIX A 인플레이스 연산(14단계 보충)

출력 쪽에서 전해지는 미분값을 그대로 대입하므로 같은 변수를 반복해서 사용하면 전파되는 미분값이 덮어 써지는 문제로 Variable 클래스를 다음과 같이 수정

In [20]:
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx # 다음번부터는 전달된 미분값을 더해줌
                    # x.grad += gx 라고 쓰면 문제 발생 -> 부록 A 참고
            
                if x.creator is not None:
                    funcs.append(x.creator)

1차 미분의 결과(기울기)가 전파됐을 때는 x.grad = gx로 대입하고, 그 이후로는 x.grad = x.grad + gx로 '누적'하는 부분에서  
x.grad += gx 라고 쓰면 문제가 됨

In [21]:
import numpy as np
x = np.array(1)
print(id(x)) 

x += x # 덮어 쓰기
print(id(x))

x = x + x # 복사(새로 생성)
print(id(x))

140195406241488
140195406241488
140194859145264


위 예시와 같이 누적 대입 연산자인 +=를 사용하면 x의 객체ID가 변하지 않음(메모리 위치가 동일). 즉, 값만 덮어 쓴 것임  
이처럼 복사하지 않고 메모리의 값을 직접 덮어 쓰는 연산을 인플레이스 연산(in-place operation)이라고 함  
  
한편 x = x + x를 실행했을 때는 객체ID가 달라졌음  
다른 위치에 새로운 ndarray 인스턴스가 생성(복사)됨  
  
메모리 효율 측면에서는 인플레이스 연산을 사용하는게 바람직할 수 있으나,  
역전파에서 문제가 생김  
  
인플레이스 연산으로 덮어쓰는 방식으로 진행하면 다음과 같다.

In [22]:
# 잘못된 예시
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
#                     x.grad = x.grad + gx 
                    x.grad += gx # 문제 발생함
            
                if x.creator is not None:
                    funcs.append(x.creator)

In [23]:
x = Variable(np.array(3))
y = add(x,x)
y.backward()

print('y.grad: {}({})'.format(y.grad, id(y.grad))) # 1
print('x.grad: {}({})'.format(x.grad, id(x.grad)))

y.grad: 2(140195404855312)
x.grad: 2(140195404855312)


인플레이스 연산이 갚을 덮어 썼기 때문에 y.grad와 x.grad가 같은 값을 참조하게 되어 잘못된 결과가 나옴.  
따라서 x.grad = x.grad + gx로 사용해야 함

In [24]:
# 바른 예시
class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray): 
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) 
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad = gx 
                else:
                    x.grad = x.grad + gx 
#                     x.grad += gx # 문제 발생함
            
                if x.creator is not None:
                    funcs.append(x.creator)

In [25]:
x = Variable(np.array(3))
y = add(x,x)
y.backward()

print('y.grad: {}({})'.format(y.grad, id(y.grad))) # 1
print('x.grad: {}({})'.format(x.grad, id(x.grad)))

y.grad: 1(140194859328976)
x.grad: 2(140194859143632)
